In [1]:
import pickle
import torch

from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam


/home/will/anaconda3/envs/pytorch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/will/anaconda3/envs/pytorch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [17]:
# model parameters
reuse = False
block = schnet_block
head = cat3Head
data = '../Data/{}_data_ACSF.pickle'
batch_size = 32
dim = 128
epochs = 5
clip = 2
layer1 = 2
layer2 = 2
factor = 2

In [12]:
train_dl,val_dl,test_dl = get_data(data,batch_size)

In [65]:
model = GNN(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')

In [66]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=1e-4)

In [69]:
model,train_loss_perType,val_loss_perType = train(opt,model,epochs,train_dl,val_dl,paras,clip)

epoch:0, train_loss: +1.097, val_loss: +0.776, 
train_vector: +4.47|+1.55|+0.59|+0.40|+0.14|+0.84|+0.89|-0.10, 
val_vector  : +4.35|+0.87|+0.40|-0.12|-0.37|+0.82|+0.66|-0.39

epoch:1, train_loss: +0.613, val_loss: +0.593, 
train_vector: +4.07|+0.77|+0.20|-0.25|-0.52|+0.69|+0.53|-0.59, 
val_vector  : +3.81|+0.69|+0.25|-0.34|-0.57|+0.87|+0.54|-0.50

epoch:2, train_loss: +0.333, val_loss: +0.047, 
train_vector: +3.00|+0.53|+0.05|-0.45|-0.73|+0.62|+0.40|-0.76, 
val_vector  : +1.58|+0.08|-0.03|-0.55|-0.83|+0.61|+0.33|-0.81

epoch:3, train_loss: +0.022, val_loss: -0.017, 
train_vector: +1.25|+0.35|-0.04|-0.57|-0.85|+0.58|+0.33|-0.88, 
val_vector  : +1.15|+0.19|-0.03|-0.68|-0.89|+0.68|+0.33|-0.88

epoch:4, train_loss: -0.091, val_loss: -0.172, 
train_vector: +0.96|+0.24|-0.12|-0.69|-0.96|+0.54|+0.26|-0.97, 
val_vector  : +1.01|+0.03|-0.20|-0.83|-1.07|+0.53|+0.21|-1.06

Training completed in 186.4336326122284s


In [170]:
save_results(train_loss_perType,val_loss_perType,reuse,block,\
             head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)

In [174]:
save_model(model,opt,reuse,block,head,data,batch_size,dim,clip,layer1,layer2,factor,epochs)

In [45]:
checkpoint = torch.load('../Model/gnn_int_logloss_2Edges.tar')
model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])

In [109]:
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'epochs':epochs
            }, '../Model/gnn_int_logloss_2Edges_ACSF.tar')

### make submissions

In [110]:
model.eval()
yhat_list = []
with torch.no_grad():
    for data in test_dl:
        data = data.to('cuda:0')
        yhat_list.append(model(data,False))

yhat = torch.cat(yhat_list).cpu().detach().numpy()
submission = pd.read_csv('../Data/sample_submission.csv')
submission['scalar_coupling_constant'] = yhat
submission.to_csv('../Submission/gnn_int_logloss_2Edges_ACSF.csv',index=False)